In [6]:
import pandas as pd
import numpy as np
import os
import enum
import time
import matplotlib.pyplot as plt
import json
import seaborn as sns
import ignite

from tqdm.auto import tqdm, trange

import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import torchio as tio
import monai

import nilearn as nl
import nibabel as nib
import nilearn.plotting as nlplt
from pathlib import Path

from torch.utils.tensorboard import SummaryWriter
from torch.utils.data import DataLoader, Dataset

from sklearn.model_selection import LeaveOneOut, train_test_split
from sklearn.metrics import precision_recall_curve

from matplotlib.colors import ListedColormap

from datetime import datetime
from functools import partialmethod

# from google.colab import drive

torch.manual_seed(7)
torch.set_grad_enabled(False)
# os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:64"

device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")

torch.set_grad_enabled(False)
figsize = 16, 9
tio.Subject.plot = partialmethod(tio.Subject.plot, figsize=figsize, reorient=False)

In [5]:
study_path = "../../study"

info_df = pd.read_csv(f"{study_path}/stats/info.csv").dropna()

subjects_list = []

for idx, row in info_df.iterrows():
    if "VNSLC" not in row.ID:
        continue
    
    subj_dict = {
        "id": row.ID,
        "resp": row.resp, # Prova cambiando in 0, 0.75, 1
        # "age": row.age,
        # "sex": row.sex,
        # "epilepsy_type": row.epilepsy_type,
        # "epilepsy_onset_age": row.epilepsy_onset_age,
        # "therapy_duration": row.therapy_duration,
        # "AEDs": row.AEDs,
        "FA": tio.ScalarImage(f"{study_path}/subjects/{row.ID}/registration/FAinMNI_affine/{row.ID}_FA.nii.gz"),
        "MD": tio.ScalarImage(f"{study_path}/subjects/{row.ID}/registration/FAinMNI_affine/{row.ID}_MD.nii.gz"),
        "AD": tio.ScalarImage(f"{study_path}/subjects/{row.ID}/registration/FAinMNI_affine/{row.ID}_AD.nii.gz"),
        "RD": tio.ScalarImage(f"{study_path}/subjects/{row.ID}/registration/FAinMNI_affine/{row.ID}_RD.nii.gz"),
        "wFA": tio.ScalarImage(f"{study_path}/subjects/{row.ID}/registration/FAinMNI_affine/{row.ID}_diamond_wFA.nii.gz"),
        "wMD": tio.ScalarImage(f"{study_path}/subjects/{row.ID}/registration/FAinMNI_affine/{row.ID}_diamond_wMD.nii.gz"),
        "wAD": tio.ScalarImage(f"{study_path}/subjects/{row.ID}/registration/FAinMNI_affine/{row.ID}_diamond_wAD.nii.gz"),
        "wRD": tio.ScalarImage(f"{study_path}/subjects/{row.ID}/registration/FAinMNI_affine/{row.ID}_diamond_wRD.nii.gz"),
        # "diamond_frac_csf": tio.ScalarImage(f"{study_path}/subjects/{row.ID}/registration/FAinMNI_affine/{row.ID}_diamond_frac_csf.nii.gz"),
        "icvf": tio.ScalarImage(f"{study_path}/subjects/{row.ID}/registration/FAinMNI_affine/{row.ID}_noddi_icvf.nii.gz"),
        "odi": tio.ScalarImage(f"{study_path}/subjects/{row.ID}/registration/FAinMNI_affine/{row.ID}_noddi_odi.nii.gz"),
        "fextra": tio.ScalarImage(f"{study_path}/subjects/{row.ID}/registration/FAinMNI_affine/{row.ID}_noddi_fextra.nii.gz"),
        "fiso": tio.ScalarImage(f"{study_path}/subjects/{row.ID}/registration/FAinMNI_affine/{row.ID}_noddi_fiso.nii.gz"),
        "wfvf": tio.ScalarImage(f"{study_path}/subjects/{row.ID}/registration/FAinMNI_affine/{row.ID}_mf_wfvf.nii.gz"),
        "fvf_tot": tio.ScalarImage(f"{study_path}/subjects/{row.ID}/registration/FAinMNI_affine/{row.ID}_mf_fvf_tot.nii.gz"),
        # "mf_frac_csf": tio.ScalarImage(f"{study_path}/subjects/{row.ID}/registration/FAinMNI_affine/{row.ID}_mf_frac_csf.nii.gz"),
        # "aparc_aseg": tio.LabelMap(f"{study_path}/freesurfer/{row.ID}/registration/FAinMNI_affine//aparc+aseg+thalnuc.bbr.nii.gz"),
        # "t1": tio.ScalarImage(f"{study_path}/subjects/{row.ID}/registration/FAinMNI_affine/{row.ID}_T1_brain_reg.nii.gz"),
    }

    subjects_list.append(tio.Subject(subj_dict))